In [1]:
import os
from pathlib import Path

import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
import numpy as np


In [2]:
print("Current working directory:")
print(os.getcwd())

# If notebook is inside /notebooks, move one level up
if Path(os.getcwd()).name == "notebooks":
    os.chdir("..")

print("\nUpdated working directory:")
print(os.getcwd())


Current working directory:
c:\Users\zahee\Videos\soil-carbon-proxy-mrv\notebooks

Updated working directory:
c:\Users\zahee\Videos\soil-carbon-proxy-mrv


In [3]:
print("NDVI:", list(Path("data/raw/ndvi").glob("*")))
print("BSI:", list(Path("data/raw/bsi").glob("*")))
print("SAVI:", list(Path("data/raw/savi").glob("*")))
print("WorldCover:", list(Path("data/raw/worldcover").glob("*")))


NDVI: [WindowsPath('data/raw/ndvi/NDVI_Summer_JJA_2024_merged.tif')]
BSI: [WindowsPath('data/raw/bsi/BSI_Summer_JJA_2024_merged.tif')]
SAVI: [WindowsPath('data/raw/savi/SAVI_Summer_JJA_2024_merged.tif')]
WorldCover: [WindowsPath('data/raw/worldcover/WorldCover_CroplandMask_Brandenburg.tif')]


In [4]:
ndvi_path = Path("data/raw/ndvi/NDVI_Summer_JJA_2024_merged.tif")
bsi_path  = Path("data/raw/bsi/BSI_Summer_JJA_2024_merged.tif")
savi_path = Path("data/raw/savi/SAVI_Summer_JJA_2024_merged.tif")

wc_path = Path("data/raw/worldcover/WorldCover_CroplandMask_Brandenburg.tif")

out_dir = Path("data/processed/cropland_only")
out_dir.mkdir(parents=True, exist_ok=True)


In [5]:
for p in [ndvi_path, bsi_path, savi_path, wc_path]:
    print(p.name, "exists:", p.exists())


NDVI_Summer_JJA_2024_merged.tif exists: True
BSI_Summer_JJA_2024_merged.tif exists: True
SAVI_Summer_JJA_2024_merged.tif exists: True
WorldCover_CroplandMask_Brandenburg.tif exists: True


In [6]:
with rasterio.open(wc_path) as wc_src:
    wc_data = wc_src.read(1)
    wc_transform = wc_src.transform
    wc_crs = wc_src.crs

print("WorldCover loaded")
print("Unique values:", np.unique(wc_data))


WorldCover loaded
Unique values: [0 1]


In [7]:
def apply_cropland_mask(input_raster, output_raster,
                        wc_data, wc_transform, wc_crs):

    with rasterio.open(input_raster) as src:
        data = src.read(1)
        meta = src.meta.copy()

        # Prepare array for resampled WorldCover
        wc_resampled = np.zeros(data.shape, dtype=np.uint8)

        reproject(
            source=wc_data,
            destination=wc_resampled,
            src_transform=wc_transform,
            src_crs=wc_crs,
            dst_transform=src.transform,
            dst_crs=src.crs,
            resampling=Resampling.nearest
        )

        # Apply cropland mask
        masked = np.where(wc_resampled == 1, data, np.nan)

    meta.update({
        "dtype": "float32",
        "nodata": np.nan
    })

    with rasterio.open(output_raster, "w", **meta) as dst:
        dst.write(masked.astype("float32"), 1)

    print(f"Saved: {output_raster.name}")


In [8]:
apply_cropland_mask(
    ndvi_path,
    out_dir / "NDVI_Summer_JJA_2024_cropland.tif",
    wc_data, wc_transform, wc_crs
)

apply_cropland_mask(
    bsi_path,
    out_dir / "BSI_Summer_JJA_2024_cropland.tif",
    wc_data, wc_transform, wc_crs
)

apply_cropland_mask(
    savi_path,
    out_dir / "SAVI_Summer_JJA_2024_cropland.tif",
    wc_data, wc_transform, wc_crs
)


Saved: NDVI_Summer_JJA_2024_cropland.tif
Saved: BSI_Summer_JJA_2024_cropland.tif
Saved: SAVI_Summer_JJA_2024_cropland.tif


In [9]:
def count_valid_pixels(path):
    with rasterio.open(path) as src:
        data = src.read(1)
    return np.sum(~np.isnan(data))

for f in out_dir.glob("*.tif"):
    print(f.name, "valid pixels:", count_valid_pixels(f))


BSI_Summer_JJA_2024_cropland.tif valid pixels: 138240312
NDVI_Summer_JJA_2024_cropland.tif valid pixels: 138240312
SAVI_Summer_JJA_2024_cropland.tif valid pixels: 146290106
